In [19]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from AdaBandFlt import *
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

%matplotlib tk

In [4]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

In [5]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

xminnoise = int(np.round(11114*(fs/1000)))
xmaxnoise = int(np.round(18511*(fs/1000)))

noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

xminspike = int(np.round(130826*(fs/1000)))
xmaxspike = int(np.round(131699*(fs/1000)))

burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

begin_data = signal_filtered.iloc[:500000]

In [6]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL (PASSE-BANDE+WIENER)

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - spike_data : part of signal where burst (after bandpass)


signal = signal_filtered

In [7]:
fs = 25000

noise_levels = init_noise_levels(signal_filtered, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)


In [8]:
plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5, 1.0, 'Noise Levels')

In [9]:
spike_info = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.0005, 
                           noise_window_size = 0.01,
                           threshold_factor = 3.4,
                           maxseparation = 0.0015)

spike_info.columns = ['indice_max','indice_min','i_max-i_min','Delta_amplitudes']
spike_info

,indice_max,indice_min,i_max-i_min,Delta_amplitudes
0,53256,53240,16,14.278395
1,53278,53286,-8,16.236913
2,53391,53382,9,18.693860
3,53492,53504,-12,31.926263
4,53517,53504,13,30.926425
...,...,...,...,...
572,477952,477961,-9,33.708894
573,478147,478158,-11,32.536774
574,478169,478158,11,27.879169
575,478305,478290,15,16.864888


In [10]:
spike_info = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.0005, 
                           noise_window_size = 0.01,
                           threshold_factor = 3.4,
                           maxseparation = 0.0015)

spike_info.columns = ['indice_max','indice_min','i_max-i_min','Delta_amplitudes']
spike_info

,indice_max,indice_min,i_max-i_min,Delta_amplitudes
0,53256,53240,16,14.278395
1,53278,53286,-8,16.236913
2,53391,53382,9,18.693860
3,53492,53504,-12,31.926263
4,53517,53504,13,30.926425
...,...,...,...,...
572,477952,477961,-9,33.708894
573,478147,478158,-11,32.536774
574,478169,478158,11,27.879169
575,478305,478290,15,16.864888


In [11]:
spike_data = record_spikes(signal, fs, spike_info.iloc[:,0], 
                  t_before = 0.0008, 
                  t_after = 0.0008)

initial index:0
(578, 41)


In [12]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_info.iloc[:,0],
                  t_before = 0.0008,
                  t_after = 0.0008)

In [13]:
#signal.plot(color = 'blue')
#spike_data_oneline.plot(color = 'red')
plt.plot(df.index, signal, color = 'blue')
plt.plot(spike_data_oneline.index, spike_data_oneline, color = 'red')
plt.title('Filtered Signal with Detected Spikes with RMS')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.legend()
plt.grid(True)

No handles with labels found to put in legend.


In [14]:
plt.figure(figsize = (100,20))
#plt.plot(spike_data)
spike_data.iloc[:,9:200].plot()
plt.xlabel('Time')
plt.grid()

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [20]:
pca_data = np.array(spike_data.iloc[:,1:].values).transpose()

In [ ]:
#y = [0 for i in range(len(pca_data))]
#i=0
#for data in pca_data:
#    if data[25]>0:
#        y[i]=1
#    i+=1
    
fig = plt.figure(4, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

plt.cla()
pca = PCA(n_components=3)
pca.fit(pca_data)
X = pca.transform(pca_data)

# Reorder the labels to have colors matching the cluster results
y = np.choose(y, [1, 0]).astype(np.float)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y, cmap=plt.cm.nipy_spectral,
           edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

plt.show()